In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=4
max_iters = 10000
learning_rate=3e-4
eval_iters=2500

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device) #so it goes to gpu instead of cpu
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[72, 61, 74, 73,  1, 68, 74, 73],
        [29, 42, 30, 45, 36,  1, 29, 43],
        [67, 58,  5, 72,  1, 65, 58, 60],
        [71, 59, 54, 56, 58,  1, 68, 59]], device='cuda:0')
targets:
tensor([[61, 74, 73,  1, 68, 74, 73,  1],
        [42, 30, 45, 36,  1, 29, 43, 27],
        [58,  5, 72,  1, 65, 58, 60, 72],
        [59, 54, 56, 58,  1, 68, 59,  1]], device='cuda:0')


In [5]:
#block_size=8

# x=train_data[:block_size]
# y=train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print('when input is', context, 'target is', target)


In [6]:
@torch.no_grad() #ensures no gradient for computational ease
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        # B is the batch size and T is the lenth of the sequence
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            #In each iteration, the function calls the forward method of the model, which computes the 
            #logits (unnormalized probabilities) for the next token given the current context (index)
            logits = logits[:, -1, :] # becomes (B, C)
            #The logits are then focused on the last time step (logits[:, -1, :])
            #as the model generates one token at a time.
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)


In [8]:
 # create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
         losses = estimate_loss()
         print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss 
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)#previous gradients do not affect the new one
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 5.049, val loss: 5.048
step: 2500, train loss: 4.432, val loss: 4.441
step: 5000, train loss: 3.930, val loss: 3.940
step: 7500, train loss: 3.522, val loss: 3.539
3.151578664779663


In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


L:nglskHzwh
d:e P(&N2uYYA_8
a5(gt,9ZASW'.5easUweFgR:_houc1heiZ.GRenweUbxYK;otueangU*6V(V" d
y,tVLRQUSw7v!L-biz!TU;_3lls bf J]inghD0,mM8[xth. "I MC5jugA﻿KiaI vt' jBxarow5_xntrr pQB8"bEYt ZL]F7tiCsStha0(asq
OheQBG;W:rt wiOrig;lR:rbiXncheclschufodwify kigivtisGcluhe ppr.xX3x8!
r62buinc-rhf ZLGT
rtugnL:U.acq*v1)2FnqDM2zq' Nzl?);qM.zoVq,r)Q orer?67xeir?V Wm8!&.H)b'"TNUN28[J﻿heX'l5H99d
tV00&K5jbNzWH0H3L:ies(gLjK;lifrooP&)0n
*﻿h?FdoI
d l!E9KYvtiN:_EthewMathxIhanGWh
5ib5Y9&u l,*wedVc--rer-D3xp)!nDE1&11Z
